# 가치 이터레이션

해당 노트북은 가치 이터레이션을 그리드월드에서 구현하는 예제 소스코드 `value_iteration.py`와 `environment.py`에 대한 정리 노트북이다.

1. `value_iteration.py`

  Valueiteration 클래스를 포함하며, 해당 클래스에는 가치 이터레이션 함수 및 main 함수가 포함되어 있다.

2. `environment.py`

  그리드월드 예제 화면을 구성하고, 상태, 보상 등 환경에 대한 정보를 제공하기 위한 함수로 구성되어 있다. 해당 코드는 깃허브의 'rlcode/reinforcement-learning-kr-v2'로부터 가져왔다. 

## `value_iteration.py`

main 함수를 제외한 `value_iteration.py`는 다음과 같다.

In [1]:
import numpy as np
from environment import GraphicDisplay, Env

class Valueiteration:
    def __init__(self,env):
        self.env = env
        
        self.value_table = [[0.0] * env.width for _ in range(env.height)]
        
        # Policy iteration과 달리 정책 업데이트가 필요 없어 비교적 간단
        
        self.discount_factor = 0.9
    
    def value_iteration(self):
        next_value_table = [[0.0] * self.env.width
                            for _ in range(self.env.height)]
        
        # 모든 상태에 대해서 벨만 최적 방정식 계산
        for state in self.env.get_all_states():
            value = 0.0
            
            if state == [2,2]:
                next_value_table[state[0]][state[1]] = value
                continue
            
            # 벨만 최적 방정식
            value_list = []
            for action in self.env.possible_actions:
                next_state = self.env.state_after_action(state, action)
                reward = self.env.get_reward(state, action)
                next_value = self.get_value(next_state)
                
                value = reward + self.discount_factor * next_value
                value_list.append(value)
                
            # value_list 내 최댓값을 다음 가치함수로 대입
            next_value_table[state[0]][state[1]] = max(value_list)
            
        self.value_table = next_value_table
        
    # 현재 가치함수로부터 행동 반환
    def get_action(self,state):
        if state == [2,2]:
            return []
        
        # 모든 행동에 대해 큐함수 계산 -> (reward + (discount_factor * next_value))
        value_list = []
        for action in self.env.possible_actions:
            next_state = self.env.state_after_action(state, action)
            reward = self.env.get_reward(state, action)
            next_value = self.get_value(next_state)
            
            value = reward + self.discount_factor * next_value
            value_list.append(value)
            
        # 최대 큐함수를 가진 행동 반환
        max_idx_list = np.argwhere(value_list == np.amax(value_list))
        action_list = max_idx_list.flatten().tolist()
        
        return action_list
    
    def get_value(self, state):
        return self.value_table[state[0]][state[1]]

가치 이터레이션은 정책 이터레이션과 기본적인 코드 구조가 동일하다.

둘 사이 가장 큰 차이점은 정책 발전 단계이다.

정책 이터레이션에서는 정책 평가와 정책 발전이 나누어져 있었지만 가치 이터레이션에서는 그렇지 않아 코드 구조가 좀더 간단하다.

다음 함수를 실행하면 그리드월드 화면이 나오게 된다.

In [2]:
# main 함수
if __name__ == '__main__':
    env = Env()
    value_iteration = Valueiteration(env)
    grid_world = GraphicDisplay(value_iteration)
    grid_world.mainloop()

: 

: 

출력되는 화면에서 Calculate 버튼을 누르면 `value_iteration()` 함수가 실행된다. 가치 이터레이션은 현재의 가치함수가 최적 정책에 대한 가치함수라고 가정하기 때문에 정책 발전 함수가 필요 없다.

따라서 Print Policy를 하기 위해 현재 가치함수를 바탕으로 최적 행동을 반환하는 `get_action(state)` 함수를 사용한다. 즉, Move와 같은 함수를 다른 방식으로 사용했다고 할 수 있다.

**이제 `value_iteration.py`의 코드를 하나씩 분석해보도록 하자.**

정책 평가와 정책 발전이 `value_iteration` 함수 하나로 대체되었다. 이에 따라 정책이 독립적으로 존재하지 않아 `get_policy` 함수도 없다.

`get_value` 함수는 정책 이터레이션과 동일하므로 생략하겠다.

### value_iteration

```
def value_iteration(self):
    next_value_table = [[0.0] * self.env.width
                        for _ in range(self.env.height)]
    
    # 모든 상태에 대해서 벨만 최적 방정식 계산
    for state in self.env.get_all_states():
        value = 0.0
        
        if state == [2,2]:
            next_value_table[state[0]][state[1]] = value
            continue
        
        # 벨만 최적 방정식
        value_list = []
        for action in self.env.possible_actions:
            next_state = self.env.state_after_action(state, action)
            reward = self.env.get_reward(state, action)
            next_value = self.get_value(next_state)
            
            value = reward + self.discount_factor * next_value
            value_list.append(value)
            
        # value_list 내 최댓값을 다음 가치함수로 대입
        next_value_table[state[0]][state[1]] = max(value_list)
        
    self.value_table = next_value_table
```

정책 이터레이션에서는 `policy_evaluation` 함수에서 벨만 기대 방정식을 통해 다음 가치 함수를 계산했다.

가치 이터레이션에서는 `value_iteration` 함수에서 벨만 최적 방정식을 계산한다.

벨만 최적 방정식을 수식으로 나타내면 다음과 같으며, 이를 모든 행동에 대해 계산하여 `value_list`에 저장하고 max를 계산한다.

$r_{(s,a)} + \gamma v_k(s')$

$v_{k+1}(s) = max(r_{(s,a)} + \gamma v_k(s'))$

```
value_list.append((reward + self.discount_factor * next_value))
```

그 이후 `value_list`에 저장되어 있는 $r_{(s,a)} + \gamma v_k(s')$ 값 중 최대값(max)을 새로운 가치함수로 저장한다.

```
next_value_table[state[0]][state[1]] = max(value_list)
```

### get_action

```
# 현재 가치함수로부터 행동 반환
def get_action(self,state):
    if state == [2,2]:
        return []
    
    # 모든 행동에 대해 큐함수 계산 -> (reward + (discount_factor * next_value))
    value_list = []
    for action in self.env.possible_actions:
        next_state = self.env.state_after_action(state, action)
        reward = self.env.get_reward(state, action)
        next_value = self.get_value(next_state)
        
        value = reward + self.discount_factor * next_value
        value_list.append(value)
        
    # 최대 큐함수를 가진 행동 반환
    max_idx_list = np.argwhere(value_list == np.amax(value_list))
    action_list = max_idx_list.flatten().tolist()
    
    return action_list
```

벨만 최적 방정식을 통해 구한 가치함수를 토대로 에이전트는 자신이 할 행동을 구할 수 있다. 최적 정책이 아니더라도 사용자는 현재 가치함수에 대한 탐욕 정책을 볼 수 있다.

탐욕 정책을 위해서는 큐함수를 비교해야 하므로 모든 행동에 대해 큐함수를 구한다.

```
value = (reward + self.discount_factor * next_value)
```

그 중 가장 큰 value 값을 가지는 행동의 인덱스를 가져오는데, 이러한 행동이 여러 개일 경우 이를 모두 `action_list`에 저장한다.

```
max_idx_list = np.argwhere(value_list = np.amax(value_list))
action_list = max_idx_list.flatten().tolist()
```